# Feed Forward Neural Network Regression with Keras

### Predict fuel efficiency (Copyright 2018 The TensorFlow Authors)

## Imports

In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals
import pathlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

## Load data

In [ ]:
path = keras.utils.get_file("auto-mpg.data", "http://archive.ics.uci.edu/ml/machine-learning-databases/auto-mpg/auto-mpg.data")
path

Import it using pandas

In [ ]:
column_names = ['MPG','Cylinders','Displacement','Horsepower','Weight', 'Acceleration', 'Model Year', 'Origin']
raw_ds = pd.read_csv(path, names=column_names, na_values = "?", comment='\t', sep=" ", skipinitialspace=True)
ds = raw_ds.copy()
ds.tail()

## Data cleansing

In [ ]:
ds.isna().sum()

In [ ]:
ds = ds.dropna()

The `"Origin"` column is really categorical, not numeric. So convert that to a one-hot:

In [ ]:
ds['Origin'] = ds['Origin'].map({1: 'USA', 2: 'Europe', 3: 'Japan'})

In [ ]:
ds = pd.get_dummies(ds, prefix='', prefix_sep='')
ds.tail()

In [ ]:
train_ds = ds.sample(frac=0.8,random_state=0)
test_ds = ds.drop(train_ds.index)

## Data analysis

In [ ]:
sns.pairplot(train_ds[["MPG", "Cylinders", "Displacement", "Weight"]], diag_kind="kde");

Also look at the overall statistics:

In [ ]:
stats = train_ds.describe()
stats.pop("MPG")
stats = stats.transpose()
stats

## Separate target and Normalize

In [ ]:
train_labels = train_ds.pop('MPG')
test_labels = test_ds.pop('MPG')

### Normalize the data

Look again at the `train_stats` block above and note how different the ranges of each feature are.

In [ ]:
def normalize(x):
  return (x - stats['mean']) / stats['std']

norm_train_ds = normalize(train_ds)
norm_test_ds = normalize(test_ds)

## Build Model

In [ ]:
def build_model():
  model = keras.Sequential([
    layers.Dense(64, activation='relu', input_shape=[len(train_ds.keys())]),
    layers.Dense(64, activation='relu'),
    layers.Dense(1)
  ])

  optimizer = tf.keras.optimizers.RMSprop(0.001)

  model.compile(loss='mse', optimizer=optimizer, metrics=['mae', 'mse'])
  return model

In [ ]:
model = build_model()
model.summary()

In [ ]:
example = norm_train_ds[:10]
model.predict(example)

## Train

In [ ]:
res = model.fit(norm_train_ds, train_labels, epochs=1000, validation_split = 0.2, verbose=1)

In [ ]:
hist = pd.DataFrame(res.history)
hist['epoch'] = res.epoch
hist.tail()

In [ ]:
loss, mae, mse = model.evaluate(norm_test_ds, test_labels, verbose=2)
print("Testing set Mean Abs Error: {:5.2f} MPG".format(mae))

## Predict

Finally, predict MPG values using data in the testing set:

In [ ]:
pred = model.predict(norm_test_ds).flatten()

a = plt.axes(aspect='equal')
plt.scatter(test_labels, pred)
plt.xlabel('True Values [MPG]')
plt.ylabel('Predictions [MPG]')
lims = [0, 50]
plt.xlim(lims)
plt.ylim(lims)
_ = plt.plot(lims, lims)


In [ ]:
error = pred - test_labels
sns.distplot(error);

In [16]:
from pyspark import SparkContext, SparkConf
conf = SparkConf().setAppName('Elephas_App').setMaster('local[8]')
sc = SparkContext(conf=conf)

In [17]:
from elephas.utils.rdd_utils import to_simple_rdd
rdd = to_simple_rdd(sc, norm_train_ds, train_labels)

WARNING


In [18]:
from elephas.spark_model import SparkModel

spark_model = SparkModel(model, frequency='epoch', mode='asynchronous')
spark_model.fit(rdd, epochs=20, batch_size=32, verbose=0, validation_split=0.1)

c:\program files\python38\lib\site-packages\elephas\spark_model.py:66: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  if self.mode is not 'synchronous':


>>> Fit model


ValueError: ('Could not interpret optimizer identifier:', False)

In [ ]:
import tensorflow as tf

In [ ]:
from tensorflow.keras.utils import generic_utils

## Credits & Links

https://www.tensorflow.org/tutorials/keras/regression